# DIC4/587: Phase 2
*Contributors: Seungmin Lee, David Huang*

---
## Deliverables

1. Algorithms/Visualizations: Apply 5 different significant and relevant
algorithms (ML, MR, and/or statistical models) to your data and create visualizations for
the results. 
    - For 487 students: at least 1 of the 5 algorithms must be one that was not
discussed in class. 
    - For 587 students: at least 2 must be from outside of class. 
    - These outside algorithms can come from the class textbooks, or other sources. Cite the appropriate sources for each outside algorithm you choose to apply.
2. Explanation and Analysis: For each of the 5 above algorithms, provide
justification for why you chose the particular algorithm, and discuss the effectiveness of
the algorithm when applied to your data to answer questions related to your problem
statement. 
    - This should include discussion of any relevant metrics for demonstrating
model effectiveness, as well as any intelligence you were able to gain from application of
the algorithm to your data.


## Ideas
- Incorperating word count into Naive Bayes Classification for description of what the permits were used for (ex. swimming pool reno)
- Log Regression

*Note: Disregarding driveway/sidewalks*

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
data = pd.read_csv('Dataset/ICP_Residential_Austin2017up.csv')
data.shape

(55976, 61)

In [3]:
buildData = data[data['Permit Type Desc']=="Building Permit"]
mechData = data[data['Permit Type Desc']=="Mechanical Permit"]
plumbData = data[data['Permit Type Desc']=="Plumbing Permit"]
elecData = data[data['Permit Type Desc']=="Electrical Permit"]

In [4]:
'''
Word Count Function
Input: text -> String
Output: Counter[String -> Int]
Returns a dictionary containing word counts
'''
def wc(text):
    words = re.findall("[a-z']+", text.lower().replace("<br />", ""))
    return Counter(words)